In [ ]:
!nvidia-smi

Sun Nov 17 12:44:00 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:E1:00.0 Off |                    0 |
| N/A   50C    P0             53W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# 1. Install packages

In [1]:
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install mmcv==2.0.1 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html

!pip install -qq ftfy openmim future tensorboard scikit-learn pandas
!mim install mmengine
!git clone -b main https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 639.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89991 sha256=c68a8460e21f55afdb83fdb7f18fa8d28b7490950c2ab30cb28b2f5d7ff2acd8
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:

In [ ]:
# Ensure that you run the notebook from the mmdetection directory
%cd /workspace/mmdetection

/workspace/mmdetection


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 2. Setup the dataset

In [2]:
# Download dataset from Google Drive
!gdown 16T40TdpaB8VXohm50SySREwrzbuPcJBC
!mkdir -p data/images
!unzip -qqnd data/images images.zip
!mv Train.csv Test.csv SampleSubmission.csv data/

Downloading...
From (original): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC
From (redirected): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC&confirm=t&uuid=b1e406ca-f1fc-458c-b92f-1b1b815bc02c
To: /content/mmdetection/images.zip
100% 4.29G/4.29G [00:56<00:00, 76.5MB/s]


In [3]:
!ls data

images	SampleSubmission.csv  Test.csv	Train.csv


In [6]:
## Download pretrained model weights
!mkdir -p checkpoints
!cd checkpoints && wget -nc https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth


--2024-11-20 08:17:27--  https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.48.85.230, 8.48.85.226, 8.48.85.228, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.48.85.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 883034647 (842M) [application/octet-stream]
Saving to: ‘ddq_detr_swinl_30e.pth’

ddq_detr_swinl_30e. 100%[===================>] 842.13M  31.8MB/s    in 27s     

2024-11-20 08:17:55 (31.0 MB/s) - ‘ddq_detr_swinl_30e.pth’ saved [883034647/883034647]



# 3. Training

In [4]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings,logging

from tqdm import tqdm
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,StratifiedGroupKFold
import sklearn.metrics as skm
from sklearn import preprocessing
import torch
from torch import nn
import torch.nn.functional as F
import cv2

from mmengine import Config
from mmengine.runner import Runner
import mmdet

N_SPLITS = 5
RANDOM_STATE = 41
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
DIR_DATA = 'data'
CLASSES = {'Trophozoite':0,'WBC':1}
df = pd.read_csv(f'{DIR_DATA}/Train.csv')
df['path'] = f'{DIR_DATA}/images/'+df.Image_ID
df['category_id'] = df['class'].map(CLASSES).fillna(-1)
df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,path,category_id
0,id_u3q6jdck4j.jpg,Trophozoite,1.0,712,1241,737,1270,data/images/id_u3q6jdck4j.jpg,0.0
1,id_a6cl90trri.jpg,Trophozoite,1.0,558,1566,600,1604,data/images/id_a6cl90trri.jpg,0.0
2,id_qvc2le9sm8.jpg,Trophozoite,1.0,1317,2788,1448,2914,data/images/id_qvc2le9sm8.jpg,0.0
3,id_w8xnbd5rvm.jpg,Trophozoite,1.0,925,1744,1041,1823,data/images/id_w8xnbd5rvm.jpg,0.0
4,id_6dop09rk02.jpg,NEG,1.0,0,0,0,0,data/images/id_6dop09rk02.jpg,-1.0
...,...,...,...,...,...,...,...,...,...
23525,id_vxfv34g53d.jpg,WBC,1.0,2336,2441,2537,2647,data/images/id_vxfv34g53d.jpg,1.0
23526,id_61axoyt1e6.jpg,WBC,1.0,486,713,543,770,data/images/id_61axoyt1e6.jpg,1.0
23527,id_ipmba1040m.jpg,Trophozoite,1.0,456,497,486,535,data/images/id_ipmba1040m.jpg,0.0
23528,id_r2x9t2l5cd.jpg,Trophozoite,1.0,2974,1039,3094,1165,data/images/id_r2x9t2l5cd.jpg,0.0


In [ ]:
df.describe()

,confidence,ymin,xmin,ymax,xmax,area,fold,height,width,category_id
count,23530.0,23530.000000,23530.000000,23530.000000,23530.000000,23530.000000,23530.000000,23530.000000,23530.000000,23530.000000
mean,1.0,802.795028,1292.343731,878.255249,1366.806290,8222.295198,2.020527,1873.469783,2786.126987,0.268423
std,0.0,676.145932,860.772798,704.865276,886.518521,12489.632134,1.427392,958.928827,1046.847207,0.504838
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1080.000000,1920.000000,-1.000000
25%,1.0,317.000000,685.000000,373.000000,740.000000,1230.500000,1.000000,1080.000000,1920.000000,0.000000
50%,1.0,619.000000,1107.000000,681.000000,1169.000000,3990.000000,2.000000,1080.000000,1920.000000,0.000000
75%,1.0,1016.750000,1621.000000,1108.000000,1701.000000,10070.000000,3.000000,3016.000000,4032.000000,1.000000
max,1.0,3012.000000,4051.000000,3116.000000,4155.000000,250669.000000,4.000000,3120.000000,4160.000000,1.000000


In [6]:
df.Image_ID.value_counts()

,count
Image_ID,
id_mzq1jmom69.jpg,92
id_tmtzoe9bns.jpg,85
id_7tgo99b7gy.jpg,74
id_1xpbtfb4xh.jpg,74
id_n5talfnmmk.jpg,73
...,...
id_calvvsgfft.jpg,1
id_teng746pbq.jpg,1
id_0nkc2ttx1k.jpg,1


In [ ]:
# df['area'] = (df.xmax-df.xmin)*(df.ymax-df.ymin)
# df.area.describe()

In [7]:
df['class'].value_counts()

,count
class,
Trophozoite,15838
WBC,7004
NEG,688


In [8]:
def split_data(df):
    gkf  = StratifiedGroupKFold(n_splits=N_SPLITS,shuffle=True,random_state=RANDOM_STATE)
    df['fold'] = -1
    for fold_id, (train_index, test_index) in enumerate(gkf.split(df,y=df['class'],groups=df.Image_ID)):
        df.loc[test_index,'fold'] = fold_id
    return df

def get_splits(df,fold):
    df_trn = df[df.fold!=fold].copy()
    df_val = df[df.fold==fold].copy()

    return df_trn,df_val

df = split_data(df)
df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,path,category_id,fold
0,id_u3q6jdck4j.jpg,Trophozoite,1.0,712,1241,737,1270,data/images/id_u3q6jdck4j.jpg,0.0,0
1,id_a6cl90trri.jpg,Trophozoite,1.0,558,1566,600,1604,data/images/id_a6cl90trri.jpg,0.0,1
2,id_qvc2le9sm8.jpg,Trophozoite,1.0,1317,2788,1448,2914,data/images/id_qvc2le9sm8.jpg,0.0,0
3,id_w8xnbd5rvm.jpg,Trophozoite,1.0,925,1744,1041,1823,data/images/id_w8xnbd5rvm.jpg,0.0,0
4,id_6dop09rk02.jpg,NEG,1.0,0,0,0,0,data/images/id_6dop09rk02.jpg,-1.0,0
...,...,...,...,...,...,...,...,...,...,...
23525,id_vxfv34g53d.jpg,WBC,1.0,2336,2441,2537,2647,data/images/id_vxfv34g53d.jpg,1.0,4
23526,id_61axoyt1e6.jpg,WBC,1.0,486,713,543,770,data/images/id_61axoyt1e6.jpg,1.0,4
23527,id_ipmba1040m.jpg,Trophozoite,1.0,456,497,486,535,data/images/id_ipmba1040m.jpg,0.0,2
23528,id_r2x9t2l5cd.jpg,Trophozoite,1.0,2974,1039,3094,1165,data/images/id_r2x9t2l5cd.jpg,0.0,4


In [9]:
df['split'] = 'train'
df.loc[df.fold==FOLD,'split'] = 'val'
df.split.value_counts()

,count
split,
train,18791
val,4739


In [10]:
def gen_anno(meta,split):
    annotations = []
    images = []
    anno_id = 0
    image_id = 0

    meta=meta.copy()
    for file_name,d in tqdm(meta.groupby('Image_ID')):
        path = d.path.values[0]
        im = cv2.imread(path)
        height,width,_ = im.shape
        for _,row in d.iterrows():
            # print(file_name,width,row.width,height,row.height)
            # assert width==row.width
            # assert height==row.height
            x0, y0, x1, y1 = row.xmin,row.ymin,row.xmax,row.ymax
            w = x1 - x0
            h = y1 - y0
            bbox = np.array([x0, y0, w, h]).tolist()
            area = w * h
            if area<1:
                #print('skipping',file_name,area)
                continue

            anno = dict(
                        image_id = image_id,
                        id = anno_id,
                        category_id = row.category_id,
                        bbox = bbox,
                        area = area,
                        iscrowd = 0
                    )
            anno_id += 1
            annotations.append(anno)

        images.append(dict(id=image_id, file_name=file_name,height=height,width=width))
        image_id += 1


    coco_json = dict(images=images, annotations=annotations,
                        categories=[dict(id=0,name='TRO'),dict(id=1,name='WBC')])
    path = f'data/{split}.json'
    print(f'written {len(annotations)} annotations for {len(images)} images to {path}')
    with open(path,'w', encoding='utf-8') as f:
        json.dump(coco_json,f,ensure_ascii=False)

In [11]:
gen_anno(df[df.split=='val'],'val')
### gen_anno(df[df.split=='train'],'train')
# training on complete dataset
gen_anno(df,'train')

100%|██████████| 553/553 [00:47<00:00, 11.57it/s]


written 4603 annotations for 553 images to data/val.json


100%|██████████| 2747/2747 [03:40<00:00, 12.44it/s]


written 22841 annotations for 2747 images to data/train.json


In [12]:
!ls data

images	SampleSubmission.csv  Test.csv	Train.csv  train.json  val.json


In [ ]:
cfg_path = 'configs/ddq/ddq-detr-4scale_swinl_8xb2-30e_coco.py'
cfg = Config.fromfile(cfg_path)

load_from = 'checkpoints/ddq_detr_swinl_30e.pth'

data_dir = 'data'
img_prefix = 'images'
max_epochs = 30
val_interval = 1

bs = 2
cfg.train_num_workers = num_workers = 8
cfg.train_dataloader.batch_size = bs

cfg.data_root = data_dir
cfg.work_dir = './output'
metainfo = {
    'classes': ('TRO','WBC'),
    'palette': [
        (128, 60, 30),
        (220, 20, 60),
    ]
}
cfg.dataset_type = 'CocoDataset'

num_classes = 2

cfg.num_classes = num_classes
cfg.model.bbox_head.num_classes = num_classes

cfg.train_pipeline =[
    dict(backend_args=None, type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(prob=0.5, type='RandomFlip'),

    dict(
        transforms=[
            [
                dict(
                    keep_ratio=True,
                    scales=[
                        (
                            480,
                            1333,
                        ),
                        (
                            512,
                            1333,
                        ),
                        (
                            544,
                            1333,
                        ),
                        (
                            576,
                            1333,
                        ),
                        (
                            608,
                            1333,
                        ),
                        (
                            640,
                            1333,
                        ),
                        (
                            672,
                            1333,
                        ),
                        (
                            704,
                            1333,
                        ),
                        (
                            736,
                            1333,
                        ),
                        (
                            768,
                            1333,
                        ),
                        (
                            800,
                            1333,
                        ),
                    ],
                    type='RandomChoiceResize'),
            ],
            [
                dict(
                    keep_ratio=True,
                    scales=[
                        (
                            400,
                            1200,
                        ),
                        (
                            500,
                            1200,
                        ),
                        (
                            600,
                            1200,
                        ),
                    ],
                    type='RandomChoiceResize'),
                dict(
                    allow_negative_crop=True,
                    crop_size=(
                        384,
                        600,
                    ),
                    crop_type='absolute_range',
                    type='RandomCrop'),
                dict(
                    keep_ratio=True,
                    scales=[
                        (
                            480,
                            1333,
                        ),
                        (
                            512,
                            1333,
                        ),
                        (
                            544,
                            1333,
                        ),
                        (
                            576,
                            1333,
                        ),
                        (
                            608,
                            1333,
                        ),
                        (
                            640,
                            1333,
                        ),
                        (
                            672,
                            1333,
                        ),
                        (
                            704,
                            1333,
                        ),
                        (
                            736,
                            1333,
                        ),
                        (
                            768,
                            1333,
                        ),
                        (
                            800,
                            1333,
                        ),
                    ],
                    type='RandomChoiceResize'),
            ],
        ],
        type='RandomChoice'),

    dict(type='YOLOXHSVRandomAug'),
    dict(type='Sharpness',prob=0.5),
    dict(type='AutoContrast',prob=0.5,min_mag=0.1,max_mag=1.9,level=10),
    dict(type='Rotate', level=10, min_mag=180.,max_mag=180.,prob=0.5),

    dict(type='PackDetInputs'),
    ]


cfg.train_dataloader.dataset=dict(
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='train.json',
        backend_args=None,
        data_prefix=dict(img=img_prefix),
        filter_cfg=dict(filter_empty_gt=False),
        pipeline=cfg.train_pipeline,

        type='CocoDataset')

cfg.train_dataloader.num_workers = cfg.val_dataloader.num_workers = num_workers
cfg.val_dataloader.dataset=dict(
        data_root=cfg.data_root,
        metainfo=metainfo,
        data_prefix=dict(img=img_prefix),
        ann_file='val.json',
        pipeline = cfg.test_pipeline,
        test_mode=True,
    type='CocoDataset')

cfg.test_dataloader = cfg.val_dataloader

cfg.load_from = load_from
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=val_interval)
cfg.visualizer = dict( name='visualizer',type='DetLocalVisualizer',vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])
cfg.val_evaluator = cfg.test_evaluator = dict(
    ann_file='data/val.json',
    backend_args=None,
    format_only=False,
    metric=[
        'bbox',
    ],
    type='CocoMetric')


In [ ]:
fix_seed(RANDOM_STATE)
runner = Runner.from_cfg(cfg)
runner.train()

11/17 12:44:11 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1077708736
    GPU 0: NVIDIA A100 80GB PCIe
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=comput

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


11/17 12:45:20 - mmengine - INFO - Epoch(train) [21][  50/1374]  base_lr: 2.0000e-05 lr: 2.0000e-05  eta: 4:21:20  time: 1.1454  data_time: 0.0209  memory: 30057  grad_norm: 74.9954  loss: 13.0277  loss_cls: 0.0914  loss_bbox: 0.0603  loss_iou: 0.4938  d0.loss_cls: 0.0962  d0.loss_bbox: 0.0611  d0.loss_iou: 0.5009  d1.loss_cls: 0.0941  d1.loss_bbox: 0.0605  d1.loss_iou: 0.4959  d2.loss_cls: 0.0927  d2.loss_bbox: 0.0602  d2.loss_iou: 0.4931  d3.loss_cls: 0.0925  d3.loss_bbox: 0.0603  d3.loss_iou: 0.4935  d4.loss_cls: 0.0917  d4.loss_bbox: 0.0603  d4.loss_iou: 0.4941  enc_loss_cls: 0.1001  enc_loss_bbox: 0.0642  enc_loss_iou: 0.5153  dn_loss_cls: 0.0024  dn_loss_bbox: 0.0429  dn_loss_iou: 0.3822  d0.dn_loss_cls: 0.0129  d0.dn_loss_bbox: 0.0564  d0.dn_loss_iou: 0.4797  d1.dn_loss_cls: 0.0041  d1.dn_loss_bbox: 0.0442  d1.dn_loss_iou: 0.3921  d2.dn_loss_cls: 0.0031  d2.dn_loss_bbox: 0.0429  d2.dn_loss_iou: 0.3821  d3.dn_loss_cls: 0.0022  d3.dn_loss_bbox: 0.0429  d3.dn_loss_iou: 0.3818  d4.d

DDQDETR(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (adap_padding): AdaptivePadding()
      (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (drop_after_pos): Dropout(p=0.0, inplace=False)
    (stages): ModuleList(
      (0): SwinBlockSequence(
        (blocks): ModuleList(
          (0-1): 2 x SwinBlock(
            (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): ShiftWindowMSA(
              (w_msa): WindowMSA(
                (qkv): Linear(in_features=192, out_features=576, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=192, out_features=192, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop): DropPath()
            )
        

In [ ]:
# !cp output/epoch_30.pth ./